In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
tf.__version__

'2.13.0'

### Importing Dataset

In [3]:
dataset = pd.read_csv("Churn_Modelling.csv")
X = dataset.iloc[:,3:-1].values
y = dataset.iloc[:, -1].values

In [4]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [5]:
print(y)

[1 0 1 ... 1 1 0]


### Encoding Categorical Data

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [7]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


### One-Hot Encoding 

In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(),[1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [11]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting Dataset

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

### Feature Scaling

In [13]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## ANN

In [14]:
ann = tf.keras.models.Sequential()

In [15]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

In [16]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

In [17]:
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

In [19]:
ann.compile(optimizer= 'adam', loss='binary_crossentropy', metrics=['accuracy'])

In [20]:
ann.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.5439 - accuracy: 0.7722
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4724 - accuracy: 0.7954
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4489 - accuracy: 0.7996
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4354 - accuracy: 0.8056
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4263 - accuracy: 0.8125
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4200 - accuracy: 0.8166
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4154 - accuracy: 0.8188
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4121 - accuracy: 0.8226
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4085 - accuracy: 0.8227
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4050 - accura

In [24]:
print(ann.predict(sc.transform([[1.0, 0.0, 0.0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 [==============================] - 0s 24ms/step
[[False]]


In [25]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 935us/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [26]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test,y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1520   75]
 [ 201  204]]


0.862